In [1]:
youtube_api_key = 'your_api'

In [7]:
# pip install google-api-python-client
# pip install pandas
# pip install seaborn

In [10]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [11]:
channel_id = 'UCUhFaUpnq31m6TNX2VKVSVA'
#go to channel, views source, line format and search for ?channel_id

In [12]:
youtube = build('youtube','v3',developerKey=youtube_api_key)

In [41]:
def channel_details(youtube,channel_id):
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id)
    response = request.execute()

    data = dict(Channel_name = response['items'][0]['snippet']['title'],
                Subscribers = response['items'][0]['statistics']['subscriberCount'],
                Views = response['items'][0]['statistics']['viewCount'],
                Total_Videos = response['items'][0]['statistics']['videoCount'],
                playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
                )

    return data, response


In [42]:
data,response = channel_details(youtube,channel_id)

{'Channel_name': 'carwow',
 'Subscribers': '8940000',
 'Views': '3689990051',
 'Total_Videos': '2514',
 'playlist_id': 'UUUhFaUpnq31m6TNX2VKVSVA'}

In [43]:
response

{'kind': 'youtube#channelListResponse',
 'etag': 'BHsdVbRKztHmGAjBn2tyQFm4nT0',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'QsE1jL7cu4F4Pabg98yHRIhnSw0',
   'id': 'UCUhFaUpnq31m6TNX2VKVSVA',
   'snippet': {'title': 'carwow',
    'description': "I'm Mat Watson, and you're watching Carwow.\n\nWhether you're after in-depth advice on your next new car or simply want to watch the most exciting drag races on YouTube you'll find it all right here on the world's best car channel.\n\nAnd for everything else - from comprehensive reviews, real car prices or advice you can trust - head over to Carwow.\n\nThis IS how car-changing should feel.\n",
    'customUrl': '@carwow',
    'publishedAt': '2014-01-02T16:32:13Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/zyYj5uFuT2UyjGMBrXYxCR3pPFccjGcdWIrvlm3jx2TOa6h1V7pjHvcbS6tM5HAkcHC6k9M8=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 

In [44]:
playlist_id = 'UUUhFaUpnq31m6TNX2VKVSVA'

In [68]:
def video_ids(youtube,playlist_id):
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50)
    
    response = request.execute()

    video_ids = []

    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    more_pages = True

    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = next_page_token)
        
            response = request.execute()

            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])

            next_page_token = response.get('nextPageToken')



    return video_ids

In [70]:
video_ids_list = video_ids(youtube,playlist_id)

In [79]:
def get_video_details(youtube,video_ids_list):

    all_videos_stats = []
    
    for i in range(0,len(video_ids_list),50):

        request = youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids_list[i:i+50])
        )

        response = request.execute()

        for video in response['items']:
            video_stats = dict(
                Title = video['snippet']['title'],
                Published_Date = video['snippet']['publishedAt'],
                Views = video['statistics']['viewCount'],
                Likes = video['statistics']['likeCount'],
                Comments = video['statistics']['commentCount']
            )

            all_videos_stats.append(video_stats)

    return all_videos_stats

In [80]:
all_videos_stats = get_video_details(youtube,video_ids_list)

In [82]:
len(all_videos_stats)

2515

In [88]:
data

{'Channel_name': 'carwow',
 'Subscribers': '8940000',
 'Views': '3689990051',
 'Total_Videos': '2514',
 'playlist_id': 'UUUhFaUpnq31m6TNX2VKVSVA'}

In [87]:
carwowVideos = pd.DataFrame(all_videos_stats)

In [94]:
from datetime import datetime

datetime.strptime(carwowVideos['Published_Date'][0], '%Y-%m-%dT%H:%M:%SZ')

datetime.datetime(2016, 12, 15, 13, 29, 17)

In [98]:
carwowVideos['Published_Date'] = pd.to_datetime(carwowVideos['Published_Date'], format='%Y-%m-%dT%H:%M:%SZ')

In [168]:
carwowVideos['Views'] = carwowVideos['Views'].astype(int)
carwowVideos['Likes'] = carwowVideos['Likes'].astype(int)
carwowVideos['Comments'] = carwowVideos['Comments'].astype(int)

In [169]:
carwowVideos

,Title,Published_Date,Views,Likes,Comments
0,Mercedes A45 AMG vs Honda Civic Type R Drag Ra...,2016-12-15 13:29:17,519539,2491,398
1,Ford Fiesta 2016 hatchback 360 degree test dri...,2016-12-13 11:10:53,33557,237,21
2,Jaguar F-Pace vs Land Rover Discovery Sport vs...,2016-12-10 08:51:21,761682,5422,459
3,Volkswagen Polo 2016 practicality review | Mat...,2016-12-09 16:08:38,49649,237,8
4,Volkswagen Polo 2014-2017 review | Mat Watson ...,2016-12-09 16:08:30,899346,7116,506
...,...,...,...,...,...
2510,Kia Sportage SUV 2018 infotainment and interi...,2016-04-06 13:50:18,75809,313,10
2511,Kia Sportage SUV 2018 practicality review | Ma...,2016-04-06 13:50:18,52418,351,11
2512,Kia Sportage SUV 2018 review | Mat Watson Reviews,2016-04-05 19:28:03,1089513,5927,523
2513,Mat Watson car reviews - subscribe,2016-02-17 11:30:36,2051794,8496,784


In [170]:
carwowVideos['Published_Date'][0]
# .year >= 2020
# .year()

Timestamp('2016-12-15 13:29:17')

In [216]:
dragRaces = carwowVideos[(carwowVideos['Title'].str.contains('drag race',case=False)) & (carwowVideos['Published_Date'] >= '2022-01-01')].reset_index(drop=True)

In [217]:
dragRaces['Year']= dragRaces['Published_Date'].dt.year

In [218]:
dragRaces['Bike_Video'] = 0

In [219]:
dragRaces.loc[dragRaces['Title'].str.contains('bike',case=False),'Bike_Video'] = 1
dragRaces.loc[dragRaces['Title'].str.contains('hayabusa',case=False),'Bike_Video'] = 1
dragRaces.loc[dragRaces['Title'].str.contains('moto',case=False),'Bike_Video'] = 1
dragRaces.loc[dragRaces['Title'].str.contains('ducati',case=False),'Bike_Video'] = 1

In [220]:
dragRaces['interaction']= ( dragRaces['Likes'] + dragRaces['Comments'] ) / dragRaces['Views'] * 100

In [236]:
dragRaces_aggregated = dragRaces.groupby(['Year','Bike_Video']).Title.count().reset_index().rename(columns={'Title':'Video_Count'}).merge(
    dragRaces.groupby(['Year','Bike_Video'])[['Views','Likes','Comments']].sum().reset_index().rename(columns={'Views':'Total_Views','Likes':'Total_Likes','Comments':'Total_Comments'}),
    left_on = ['Year','Bike_Video'] , right_on = ['Year','Bike_Video'] , how = 'inner'
).merge(
    dragRaces.groupby(['Year','Bike_Video'])[['Views','Likes','Comments','interaction']].mean().reset_index().rename(columns={'Views':'Average_Views','Likes':'Average_Likes','Comments':'Average_Comments'}),
    left_on = ['Year','Bike_Video'] , right_on = ['Year','Bike_Video'] , how = 'inner'
)

In [237]:
dragRaces_aggregated

,Year,Bike_Video,Video_Count,Total_Views,Total_Likes,Total_Comments,Average_Views,Average_Likes,Average_Comments,interaction
0,2022,0,92,173084068,5585337,291228,1.881349e+06,60710.184783,3165.521739,3.592172
1,2022,1,9,23265896,761365,34300,2.585100e+06,84596.111111,3811.111111,3.552126
2,2023,0,111,216319918,6384122,382976,1.948828e+06,57514.612613,3450.234234,3.452645
3,2023,1,6,18031507,478315,27663,3.005251e+06,79719.166667,4610.500000,3.938968


In [226]:
bikes_only.to_excel

,Title,Published_Date,Views,Likes,Comments,Year,Bike_Video,interaction
0,Bugatti Chiron SS v Kawasaki H2R v Ducati Pani...,2023-12-23 09:04:00,3243360,90139,4159,2023,1,2.907417
1,"F1 vs Moto GP vs 1,400hp Porsche: DRAG RACE",2023-09-23 08:04:03,1085531,46017,2106,2023,1,4.433130
2,MotoGP Bike v Rimac Nevera v Turbo Hayabusa: D...,2023-08-26 08:07:32,4133662,112991,7725,2023,1,2.920316
3,Koenigsegg v Tesla Plaid v Turbo Hayabusa: DRA...,2023-06-17 08:04:54,8605332,185671,12346,2023,1,2.301097
4,Rally Bike v Toyota Supra v GR Yaris: DRAG RACE,2023-05-16 10:05:49,763556,29523,1200,2023,1,4.023673
5,F1 Car v BMW M Bike v 911 Turbo: DRAG RACE,2023-01-08 11:05:00,200066,13974,127,2023,1,7.048174
6,Fastest Ferrari v Fastest BMW Superbike: DRAG ...,2022-11-26 09:04:00,1919488,64731,2873,2022,1,3.521981
7,BMW M5 CS v BMW M Superbike: DRAG RACE,2022-10-29 08:05:54,2878391,68019,2335,2022,1,2.444213
8,MotoGP Bike v KTM Road Bike: DRAG RACE,2022-09-24 08:06:12,5087114,135371,4695,2022,1,2.753349
9,Porsche 911 Turbo S v Suzuki Hayabusa: DRAG RACE,2022-09-03 08:05:15,1074390,43963,2122,2022,1,4.289411


In [225]:
bikes_only = dragRaces[dragRaces['Bike_Video'] == 1].reset_index(drop=True)

In [245]:
excel_writer = pd.ExcelWriter('carwow_bike_analysis.xlsx', engine='xlsxwriter')

# Write each DataFrame to a different sheet
dragRaces_aggregated.to_excel(excel_writer, sheet_name='aggregated', index=False)
bikes_only.to_excel(excel_writer, sheet_name='all_bike', index=False)

In [246]:
dragRaces_aggregated.to_excel('aggreagte.xlsx')
bikes_only.to_excel('bikes_only.xlsx')

C:\Users\kazos\AppData\Local\Temp\ipykernel_1224\1995099174.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  dragRaces_aggregated.to_excel('aggreagte.xlsx')
C:\Users\kazos\AppData\Local\Temp\ipykernel_1224\1995099174.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  bikes_only.to_excel('bikes_only.xlsx')
